In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def crearAristaGrafoNoDirigido(grafo, nodoOrigen, nodoDestino, costo):
    if nodoOrigen not in grafo:
        grafo[nodoOrigen] = {}
    (grafo[nodoOrigen])[nodoDestino] = costo
    if nodoDestino not in grafo:
        grafo[nodoDestino] = {}
    (grafo[nodoDestino])[nodoOrigen] = costo

In [3]:
#Parametros de ejecucion del programa
nombreArchivo = "Graph Data.txt"
#numeroVerticesAlto = 20
#numeroVerticesAncho = 20
#limiteInferiorCostosEntreAristasDelGrafo = 1.0
#limiteSuperiorCostosEntreAristasDelGrafo = 2.0
#diccionarioCategoriasTraficoAColores = {"Poco" : "#2ECC71", "Medio" : "#F1C40F", "Alto" : "#E74C3C"}

In [4]:
grafo = {}

inputStream = open(nombreArchivo, "r")
lineasTextoArchivo = inputStream.readlines()

for lineaTexto in lineasTextoArchivo:
    datosArista = lineaTexto.split(" ")
    nodoOrigen = int(datosArista[0])
    nodoDestino = int(datosArista[1])
    costo = float(datosArista[2])
    colorArista = datosArista[3]
    crearAristaGrafoNoDirigido(grafo, nodoOrigen, nodoDestino, costo)

inputStream.close()